In [1]:
import ipywidgets as widgets
from SPARQLWrapper import SPARQLWrapper, JSON

class HealthDM(object):
    profile = None
    profilePresent = None
    profileAbsent = None
    
    sparqlQuery = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
PREFIX mesh2015: <http://id.nlm.nih.gov/mesh/2015/>
PREFIX mesh2016: <http://id.nlm.nih.gov/mesh/2016/>
PREFIX mesh2017: <http://id.nlm.nih.gov/mesh/2017/>
PREFIX mesh2018: <http://id.nlm.nih.gov/mesh/2018/>

SELECT ?d
FROM <http://id.nlm.nih.gov/mesh>
WHERE {{
    ?d a meshv:Term .
    {{?d meshv:prefLabel ?l
     FILTER (lcase(str(?l)) = lcase('{0}'))}}
    UNION
    {{?d meshv:altLabel ?l
     FILTER (lcase(str(?l)) = lcase('{0}'))}}
}}
ORDER BY ?d
"""
    
    # @classmethod
    # def setProfile(cls, profile):
    #     cls.profile = profile
    
    @classmethod
    def createProfile(cls):
        cls.profile = widgets.Accordion(children=[])
        return cls.profile
        
    @classmethod
    def clearTerms(cls):
        cls.profilePresent = widgets.Accordion(children=[])
        cls.profileAbsent = widgets.Accordion(children=[])
        cls.profile.children = tuple([cls.profilePresent, cls.profileAbsent])
        cls.profile.set_title(0, "Sign Present")
        cls.profile.set_title(1, "Sign Absent")

    @classmethod
    def addTerm(cls, heading, description, code, present, detail1, rate1, detail2, rate2):
        wprofile = [widgets.Text(description = "MeSH Term:", value = heading),
                    widgets.Text(description = "MeSH Code:", value = code)]
        if detail1 != "#":
            fDetail = cls.formatDetail(detail1, rate1)
            wprofile.append(widgets.Text(description = fDetail[0], value = fDetail[1]))
        if detail2 != "#":
            fDetail = cls.formatDetail(detail2, rate2)
            wprofile.append(widgets.Text(description = fDetail[0], value = fDetail[1]))
        wcontent = widgets.VBox(wprofile)
        profileList = cls.profilePresent if present else cls.profileAbsent
        profileList.children = tuple(list(profileList.children) + [wcontent])
        profileList.set_title(len(profileList.children)-1, description)
        
    @classmethod
    def formatDetail(cls, detail, rate):
        detailDesc = "Detail:"
        detailValue = detail
        if detail in ["occasionally", "often", "frequently", "sometimes",
                      "rarely", "daily", "monthly", "yearly", "each day",
                      "each month", "each year", "once a day", "once a month",
                      "once a year"]:
            detailDesc = "Frequency:"
        else:
            try:
                value = int(detail)
                detailDesc = "Intensity:"
                if rate != "#":
                    detailDesc = "Frequency:"
                    detailValue = detailValue + " / " + rate
            except ValueError:
                pass
        return [detailDesc, detailValue]

    @classmethod
    def findMeshCode(cls, heading, detail1, rate1, description, detail2, rate2):
        code = "none"
        present = True
        meshHeading = heading
        
        sparql = SPARQLWrapper("http://id.nlm.nih.gov/mesh/sparql")
        sparql.setReturnFormat(JSON)

        # looking for the heading
        sparql.setQuery(cls.sparqlQuery.format(meshHeading))
        results = sparql.query().convert()
        if len(results["results"]["bindings"]) > 0:
            code = results["results"]["bindings"][0]["d"]["value"]

        if code == "none" and (heading.lower().startswith("no") or
                               heading.lower().startswith("not")):
            # looking for the heading
            present = False
            meshHeading = heading[3:] if heading.lower().startswith("no") else heading[4:]
            sparql.setQuery(cls.sparqlQuery.format(meshHeading))
            results = sparql.query().convert()
            if len(results["results"]["bindings"]) > 0:
                code = results["results"]["bindings"][0]["d"]["value"]
            
        cls.addTerm(meshHeading, description, code, present, detail1, rate1, detail2, rate2)
    
        return "#mesh_heading#" + heading + "#tree_number#" + code
    
    @classmethod
    def interfaceMain(cls, template, title, description, firstKnot):
        indexTemplate = open("template/{}.html".format(template), "r", encoding="utf-8")
        # presentationTemplate = open("template/presentation.html", "r", encoding="utf-8")

        indexResult = open("html/index.html", "w", encoding="utf-8")
        # presentationResult = open("html/Presentation.html", "w", encoding="utf-8")
        
        indexResult.write(
            indexTemplate.read().format(title=title, description=description, firstKnot=firstKnot))
        # presentationResult.write(presentationTemplate.read().format(title=title, description=description))

        indexResult.close()
        # presentationResult.close()

    @classmethod
    def interfaceKnot(cls, template, htmlName, title, description, image):
        knotTemplate = open("template/{}.html".format(template), "r", encoding="utf-8")

        knotResult = open("html/" + htmlName + ".html", "w", encoding="utf-8")
        
        knotResult.write(knotTemplate.read().format(title=title, description=description, image=image))

        knotResult.close()

HealthDM.createProfile()

Accordion()

=== Syncope of a Forklift Driver ===
Young man had a syncope while driving a forklift.
![Forklift](images/case-forklift.png)
++ -> Emergency room

=== Emergency room ===
Nurse: Doctor, please come to the Emergency room, we have a difficult case there.

Nurse: The patient is a young {man}(male), {23 years old}(aging=23). He had a {syncope} while driving a forklift. His colleagues called an ambulance and he has just arrived.

What do you want to do?
++ Make a decision
++ More information
++ Call the supervisor

=== Make a decision ===
What is the next step?
++ Get vital data
++ Offer oxygen
++ Perform an EKG
++ Get a brain scan

=== Get vital data (notice) ===
You enter the emergency room and the patient is laid on the bed, {conscious}(consciousness=yes), {oriented}(orientation), but {distressed}(suffering). {Respiratory rate= 32 mov/min}; {Heart rate= 150bpm}; {Blood pressure= 82x66mmHg}; {Oxygen saturation= 82%}.

=== Offer oxygen (notice) ===
Offer oxygen.

=== Perform an EKG (notice) ===
An EKG was obtained:
![EKG1](images/ekg.png)

=== More information (notice) ===
His colleagues found him sited inside the forklift, {without signs of trauma}(no trauma). After they took him out and laid him down, the patient woke up but still complained of {dizziness} and could not stand up.
There was {no blood} at the scene, neither other signs of trauma. The patient had {no previous syncope episodes}(no syncope=history), he {drinks alcohol occasionally}(alcohol drinking=occasionally), and {smokes 10 cigarettes a day}(smoking=10/day).

=== Get a brain scan (notice) ===
Get a brain scan.

=== Call the supervisor (notice) ===
(write the explanation here) Syncope is one of the most common complains in the emergency department.



In [2]:
%%HTML
<iframe width="100%" height="800px" src="html/index.html"><iframe>